In [3]:
from pyspark import SparkConf, SparkContext

def loadMovieNames():
    movieNames = {}
    with open("/home/jovyan/pyspark-dir/sparkcourse/sparkcourse/u.item") as f:
        for line in f:
            fields = line.split('|')
            movieNames[int(fields[0])] = fields[1]
    return movieNames

conf = SparkConf().setMaster("local").setAppName("PopularMovies")
sc = SparkContext(conf = conf)

nameDict = sc.broadcast(loadMovieNames())

lines = sc.textFile("/home/jovyan/pyspark-dir/sparkcourse/sparkcourse/u.data")
movies = lines.map(lambda x: (int(x.split()[1]), 1))
movieCounts = movies.reduceByKey(lambda x, y: x + y)

flipped = movieCounts.map( lambda x : (x[1], x[0]))
sortedMovies = flipped.sortByKey()

sortedMoviesWithNames = sortedMovies.map(lambda countMovie : (nameDict.value[countMovie[1]], countMovie[0]))

results = sortedMoviesWithNames.collect()

for result in results:
    print (result)


ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=PopularMovies, master=local) created by __init__ at <ipython-input-1-8136112ee9f5>:12 